*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

# CD Measurements - Data Analysis

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib

from scipy import integrate
import lmfit
import os

In [2]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\DNA Origami"
datalist = os.listdir(path)
print(datalist)

['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmCl_6M', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [6]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [7]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x00000176CE19D3A0>, <cdata.CData object at 0x00000176CC657280>]
(<cdata.CData object at 0x00000176CE19D3A0>, <cdata.CData object at 0x00000176CDBA90D0>, <cdata.CData object at 0x00000176CA3CEA90>, <cdata.CData object at 0x00000176CC657280>)


## Colormaps

### Heat Maps

In [12]:
plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3D surface plots

In [8]:
plot.function3d(data_all["Control_0M"].cd_df.loc[220:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


## Function Plots 

### CD-Spectra Plot

In [11]:
wave_min = 200
wave_max = 330

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmSCN_0.5M', 'GdmSCN_2M']
plot.mult_func([20, 45, 64, 80], [data_all['Control_0M'].cd_df.loc[wave_min:wave_max],  data_all['GdmCl_0.5M'].cd_df.loc[wave_min:wave_max]], swap=True,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""], marker=['', '', '', ''], linestyle=['-', ':', ':', '-'], baseline=True, 
                   y_min=[-8], y_max=[6], y_scaling=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
wave_min = 200
wave_max = 330

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmSCN_0.5M', 'GdmSCN_2M']
for key in key_list:
    plot.mult_func([20, 45, 64, 80], [data_all[key].cd_df.loc[wave_min:wave_max]], title=key, swap=True,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""], label=["20°C", "45°C", "64°C", "80°C"], marker=['', '', '', ''], linestyle=['-', ':', ':', '-'], baseline=True, 
                   y_min=[-8], y_max=[6], y_scaling=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
data=[]
for key in data_all.keys():
    data.append([data_all[key].cd_df.T])

plot.mult_func([20, 40, 55, 60, 65, 70, 90], *data, subtitle=tuple(data_all.keys()),
               title="CD-Spectra", marker=["","", "", "", "", "", ""], x_min=210
              )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Max/Min Plots


multiple probes

In [10]:
wave_min = 210
wave_max =  230


# First data you want to add to the plot
plot_data =  ana.max_wave(data_all["Control_0M"].cd_df, wave_min=wave_min, wave_max=wave_max)
plot_data = plot_data.drop(['Wavelength'], axis=1)   
plot_data.rename(columns = {'Value': 'Control_0M'}, inplace = True)



# other Data you want to add, if always same max/min values then loop would be possible
plot_data["GdmCl_4M"] = ana.max_wave(data_all["GdmCl_4M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
plot_data["Gdm2SO4_2M"] = ana.max_wave(data_all["Gdm2SO4_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["GdmSCN_2M"] 2= ana.max_wave(data_all["GdmSCN_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]


plot.mult_func(["Control_0M", "GdmCl_4M", "Gdm2SO4_2M"], [plot_data.T], subtitle=[""], title="Max. CD Value between 210nm - 230nm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

one probe

In [15]:
wave_min = 260
wave_max =  290
title = "Max. CD-Value between " + str(wave_min) + "nm - " + str(wave_max) +"nm"

for key in data_all.keys():
    plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
    plot.mult_func(["Value"], [plot_data.T], subtitle=[""], vertical_line=[melt_T[key]], title=title, label=[key, "melting Temp."])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Correlation Analyis

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2007). Two-dimensional correlation analysis useful for spectroscopy, chromatography, and other analytical measurements. Analytical Sciences, 23(2), 139–146. https://doi.org/10.2116/analsci.23.139
</div>

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2015). Techniques of two-dimensional (2D) correlation spectroscopy useful in life science research. Biomedical Spectroscopy and Imaging, 4(2), 109–127. https://doi.org/10.3233/bsi-150105
</div>

Projection Matrix

In [8]:
test = data_all["Control_0M"].cd_df.to_numpy()
test = test[47:48].T
data = data_all["Control_0M"].cd_df.T.to_numpy()
print(test.shape, data.shape )
print(test)

(16, 1) (16, 131)
[[-6.42816]
 [-6.21929]
 [-6.02504]
 [-5.96357]
 [-5.65723]
 [-5.50681]
 [-5.32867]
 [-5.2065 ]
 [-5.06574]
 [-4.50403]
 [-3.73564]
 [-2.87439]
 [-2.72282]
 [-2.52644]
 [-2.28546]
 [-2.12292]]


In [9]:
U = np.linalg.inv(np.dot(test.T, test))
R = np.dot(test, np.dot(U, test.T))
Null = np.diag([1]*16) - R

alpha = 0
data_alpha = np.dot(Null + alpha*R, data)
df = pd.DataFrame(data_alpha, dtype='float64')

plot.heatmap(df.T, x_min=[20] )



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
U = test/np.linalg.norm(test)
loading_vector = np.dot(data.T, U)

for i in range(loading_vector.shape[0]):
    if loading_vector[i] <= 0:
        loading_vector[i] = 0
data_plus = np.dot(U, loading_vector.T)

alpha = 0
data_alpha_plus = data - (1 - alpha)*data_plus
df = pd.DataFrame(data_alpha_plus, dtype='float64')

plot.heatmap(df.T, x_min=[20] )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
data  = data_all["Control_0M"].cd_df
sync, assync = ana.correlation(data, projection=True, proj_rows=[247,247])
plot.heatmap(sync, assync, x_min=[210, 210], y_min=[210,210])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Test unevenly spaced data

In [12]:
test_mat = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
test_vect = np.array([[1],[2],[3]])
print(test_vect, "\n", test_mat
     )
print(test_mat*test_vect)

[[1]
 [2]
 [3]] 
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
[[ 1  2  3]
 [ 8 10 12]
 [21 24 27]]


In [13]:
col = test.columns
T = list(col)
T = [2*T[0] - T[1]] + T + [T[-1]*2 - T[-2]]
print(np.reshape(T,(len(T), 1)))

[[ 10]
 [ 20]
 [ 30]
 [ 40]
 [ 45]
 [ 50]
 [ 53]
 [ 56]
 [ 58]
 [ 60]
 [ 62]
 [ 64]
 [ 67]
 [ 70]
 [ 75]
 [ 80]
 [ 90]
 [100]]


In [15]:
print(data_all["GdmCl_4M"].cd_df.columns)
print(data_all["GdmCl_2M"].cd_df.columns)
print(data_all["Gdm2SO4_4M"].cd_df.columns)

df = pd.concat([data_all["GdmCl_2M"].cd_df, data_all["Gdm2SO4_4M"].cd_df], join='inner' )
col = pd.concat([data_all["GdmCl_2M"].cd_df, data_all["Gdm2SO4_4M"].cd_df], join='inner').columns
print(data_all["GdmCl_4M"].cd_df.loc[:, col].shape)

Index([20, 30, 40, 45, 50, 53, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object')
Index([20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object')
Index([20, 30, 40, 45, 50, 53, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object')
(131, 14)


In [16]:
test_arr = np.reshape(np.arange(1, 131*15 + 1), (131,15))
arr = np.arange(1, 15+1)
temp_list = np.array(data_all["GdmCl_2M"].cd_df.columns)
temp_list = np.insert(temp_list, 0, 2 * temp_list[0] - temp_list[1])
temp_list = np.append(temp_list, temp_list[-1] * 2 - temp_list[-2])
test = np.array([temp_list[arr+1]]- temp_list[arr-1])
print(*test)
print(test_arr*test)

[20 20 15 10 11 8 4 4 4 5 6 8 10 15 20]
[[20 40 45 ... 130 210 300]
 [320 340 270 ... 280 435 600]
 [620 640 495 ... 430 660 900]
 ...
 [38420 38440 28845 ... 19330 29010 38700]
 [38720 38740 29070 ... 19480 29235 39000]
 [39020 39040 29295 ... 19630 29460 39300]]


In [18]:
print(temp_list)

[10 20 30 40 45 50 56 58 60 62 64 67 70 75 80 90 100]


In [20]:
# creating Hilbert-Noda-matrix for async spectrum
arr = np.arange(1, len(col) + 1, dtype=int)
h_n_m = temp_list[arr] - np.array([temp_list[arr]]).T + np.identity(len(col), dtype=int)
h_n_m = 1 / (2* np.pi * h_n_m)
h_n_m = h_n_m * np.array([temp_list[arr+1]-temp_list[arr-1]])
h_n_m = (h_n_m - h_n_m.T) / 2


[[0.0 0.3183098861837907 0.1392605752054084 0.0954929658551372
  0.08223005393081259 0.06189358898018153 0.050259455713230115
  0.047746482927568605 0.04547284088339866 0.04521447246928846
  0.044021580004141254 0.044563384065730696 0.04340589357051691
  0.046420191735136146]
 [-0.3183098861837907 0.0 0.2785211504108168 0.15915494309189535
  0.1233450808962189 0.08569881551102056 0.068209261325098
  0.06366197723675814 0.05968310365946076 0.058512846724961515
  0.05591930432958485 0.05570423008216337 0.05305164769729844
  0.05570423008216337]
 [-0.1392605752054084 -0.2785211504108168 0.0 0.3978873577297384
  0.20690142601946393 0.11439261534729978 0.08399844218738922
  0.07559859796865029 0.06872599815331845 0.06631455962162305
  0.06189358898018153 0.06100939485189322 0.05684105110424834
  0.05968310365946075]
 [-0.0954929658551372 -0.15915494309189535 -0.3978873577297384 0.0
  0.33422538049298023 0.13021768071155077 0.08569881551102056
  0.07427230677621782 0.0655343883319569 0.06282

### Homogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [38]:
vec1 = np.ones((1,15))
vec2 = np.array([np.arange(1, 16)])
print(vec1 * vec2 *test)
print(np.sum(vec1 * vec2 * test , axis=1))

[[20.0 40.0 45.0 40.0 55.0 48.0 28.0 32.0 36.0 50.0 66.0 96.0 130.0 210.0
  300.0]]
[1196.0]


In [8]:
key = "GdmCl_2M"
ref = data_all["Control_0M"].cd_df.loc[:, 20]
sync, assync = ana.correlation(data_all[key].cd_df, ref_spec=ref, center=False)

plot.heatmap(sync, assync, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210,210], y_label="wavelength[nm]", x_label="wavenlength[nm]", contour_lines=True)

[[20]
 [20]
 [15]
 [10]
 [11]
 [8]
 [4]
 [4]
 [4]
 [5]
 [6]
 [8]
 [10]
 [15]
 [20]] [10 20 30 40 45 50 56 58 60 62 64 67 70 75 80 90 100]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
for key in data_all.keys():
    if key == "Control_0M":
        continue
    ref = data_all["Control_0M"].cd_df.loc[:, 20]
    sync, assync = ana.correlation(data_all[key].cd_df, ref_spec=ref, scaling="pareto", projection=True, proj_rows=[247])
    plot.heatmap(sync, assync, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210,210], y_label="wavelength[nm]", x_label=["wavenlength[nm]"])

Index([20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object') [[20]
 [20]
 [15]
 [10]
 [11]
 [8]
 [4]
 [4]
 [4]
 [5]
 [6]
 [8]
 [10]
 [15]
 [20]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index([20, 30, 40, 45, 50, 53, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object') [[20]
 [20]
 [15]
 [10]
 [8]
 [8]
 [7]
 [4]
 [4]
 [5]
 [6]
 [8]
 [10]
 [15]
 [20]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index([20, 30, 40, 45, 50, 53, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object') [[20]
 [20]
 [15]
 [10]
 [8]
 [8]
 [7]
 [4]
 [4]
 [5]
 [6]
 [8]
 [10]
 [15]
 [20]]


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index([20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90], dtype='object') [[20]
 [20]
 [15]
 [10]
 [11]
 [8]
 [4]
 [4]
 [4]
 [5]
 [6]
 [8]
 [10]
 [15]
 [20]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index([20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 70, 75, 80, 90], dtype='object') [[20]
 [20]
 [15]
 [10]
 [11]
 [8]
 [4]
 [4]
 [4]
 [8]
 [11]
 [10]
 [15]
 [20]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Heterogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [12]:
measurement1 = "Gdm2SO4_2M"
measurement2 = "Gdm2SO4_4M"
title = "Heterogenoues 2D Correlation between " + measurement1 + " " + measurement2

ref = data_all["Control_0M"].cd_df 
sync, assync = ana.correlation(data_all[measurement1].cd_df, data_all[measurement2].cd_df, ref_spec=ref, scaling='pareto')
plot.heatmap(sync, assync, title=title, subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[220, 220], y_min=[220,220],
            y_label=measurement1 +" WL[nm]", x_label= measurement2 + " WL[nm]")

C:\Users\crazy\PycharmProjects\2Dcorrelation\hotznplots.py:80: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor="white")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sigmoid Fit and Melting Temperature

In [9]:
graph={}
melt_T={}
for key in data_all.keys():
    fit, params, error = ana.sigmoid_fit(ana.normalize(data_all[key].cd_df))
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = T
plot.mult_func([247, "fit", "deriv", "Maximum"], *graph.values(),  title="Sigmoid-Fit and Derivative(adjusted)",
               subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.18677066 61.81964508] [0.01970739 0.53616891]
[ 0.24733276 63.42770044] [0.02193106 0.35106698]
[ 0.3        63.95213937] [0.02830562 0.31740657]
[ 0.28173998 64.87467936] [0.02577176 0.3342244 ]
[ 0.2364997  63.66483834] [0.01583726 0.27656016]
[ 0.2939166  61.61989282] [0.01911078 0.21232322]
[ 0.26911012 63.19314219] [0.02238773 0.30450951]
[ 0.28883307 58.5239527 ] [0.02428439 0.29303091]
[1.41701334e-02 5.00000000e+01] [1.8215488e-02 2.3711382e+01]
[2.29854877e-02 7.00000000e+01] [ 0.01983874 16.65590283]
[ 0.19424656 57.00573805] [0.02250736 0.56124733]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot the melting Temperatures for the Gdm2SO4 and GdmCl

In [10]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Absorbance Values

In [9]:
absorbance = list(map(lambda x: data_all[x].absorb_df, list(data_all.keys())))
plot.heatmap(*absorbance, subtitle=tuple(data_all.keys()), title="Absorbance Values", x_min=[220]*len(data_all), c_min=[0]*len(data_all), c_max=[0.5]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
key = "GdmCl_6M"
fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df.loc[:,:67]), wave=260, a_range=[0.2, 0.5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph[key] = [fit]  
melt_T[key] = params[1]
error_a[key] = std[0]
error_b[key] = std[1]
coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph["GdmCl_6M"],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.5        52.63142205] [0.0834194  0.33123171]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
for key in data_all.keys():
    fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = params[1]
    error_a[key] = std[0]
    error_b[key] = std[1]
    coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph["GdmCl_4M"],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.41163539 64.15020882] [0.02314436 0.15271522]
[ 0.36215912 65.86173799] [0.02185816 0.18932737]
[ 0.44493928 65.49468303] [0.03352761 0.19683403]
[ 0.41765738 66.38498811] [0.02650862 0.17467372]
[ 0.41177395 65.56418843] [0.01837304 0.12537056]
[ 0.36338404 63.19755227] [0.02700207 0.21562398]
[ 0.38244298 64.79604632] [0.03395227 0.28041146]
[ 0.3882771  59.95701763] [0.02664735 0.18265845]
[ 0.2 50. ] [0.29051834 8.40098278]
[ 0.2        63.29561482] [0.04565207 1.08081709]
[ 0.2        62.18622927] [0.06441774 1.46286914]
[ 0.39852856 59.86735295] [0.04122087 0.2420659 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_b, title="Melting Temperature",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[°C]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
x = [0., 0.5, 2., 4.]
y = list(coop.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [coop["Control_0M"], coop["GdmCl_0.5M"], coop["GdmCl_2M"], coop["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_a, title="Variance",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K^2]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### LM Fit

In [8]:
# Parameter
sample = "Control_0M"
melt_T = {}
error_T = {}
residual = pd.DataFrame(index=["R-Wert"])
for key in data_all.keys():
    fit, params, params_error = ana.lm_fit(data_all[key].absorb_df, guess=[0.13, 0.36, 0.4])
    melt_T[key] = params['xc']
    error_T[key] = params_error
    residual[key] = [1 - (np.sum(fit["residual"]**2)/np.sum((fit[260] - fit[260].mean(axis=0))**2))]
    plot.mult_func([260, "fit"], [fit], error={"fit": fit["error"]}, swap=True, 
                   x_label="Temperature [°C]", y_label="Absorbance [%]", label=["data", "sigmoid fit", 'melting temperature'], marker=['o', ''], linestyle=[" ", "-"],
                  subtitle=["Hyperchromatic shift"], title=key, vertical_line=[params['xc']])

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 16
    # variables        = 4
    chi-square         = 8.2538e-05
    reduced chi-square = 6.8781e-06
    Akaike info crit   = -186.797542
    Bayesian info crit = -183.707187
[[Variables]]
    a:   0.13153960 +/- 0.00186326 (1.42%) (init = 0.13)
    y0:  0.35994037 +/- 0.00101406 (0.28%) (init = 0.36)
    k:   0.46427885 +/- 0.02562654 (5.52%) (init = 0.4)
    xc:  64.0734366 +/- 0.13972373 (0.22%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.618
    C(a, k)   = -0.528
    C(y0, k)  =  0.374
    C(y0, xc) =  0.283
    C(a, xc)  =  0.228
    C(k, xc)  = -0.212


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 15
    # variables        = 4
    chi-square         = 1.3729e-04
    reduced chi-square = 1.2481e-05
    Akaike info crit   = -166.021914
    Bayesian info crit = -163.189713
[[Variables]]
    a:   0.15453355 +/- 0.00274112 (1.77%) (init = 0.13)
    y0:  0.38110761 +/- 0.00145979 (0.38%) (init = 0.36)
    k:   0.39130613 +/- 0.02400150 (6.13%) (init = 0.4)
    xc:  65.4725786 +/- 0.19088378 (0.29%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.612
    C(a, k)   = -0.554
    C(y0, k)  =  0.405
    C(y0, xc) =  0.303
    C(a, xc)  =  0.248
    C(k, xc)  = -0.208


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 36
    # data points      = 15
    # variables        = 4
    chi-square         = 1.8461e-04
    reduced chi-square = 1.6782e-05
    Akaike info crit   = -161.580034
    Bayesian info crit = -158.747833
[[Variables]]
    a:   0.15740544 +/- 0.00292954 (1.86%) (init = 0.13)
    y0:  0.39680299 +/- 0.00158938 (0.40%) (init = 0.36)
    k:   0.49654664 +/- 0.03609880 (7.27%) (init = 0.4)
    xc:  65.1436370 +/- 0.18224215 (0.28%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.601
    C(a, k)   = -0.498
    C(y0, k)  =  0.344
    C(y0, xc) =  0.287
    C(a, xc)  =  0.223
    C(k, xc)  = -0.197


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 632
    # data points      = 15
    # variables        = 4
    chi-square         = 0.00210323
    reduced chi-square = 1.9120e-04
    Akaike info crit   = -125.084989
    Bayesian info crit = -122.252788
[[Variables]]
    a:   31.8133881 +/- 148.541436 (466.91%) (init = 0.13)
    y0: -0.70786515 +/- 0.58698409 (82.92%) (init = 0.36)
    k:   0.01418524 +/- 0.00851446 (60.02%) (init = 0.4)
    xc:  253.331579 +/- 443.623330 (175.12%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, xc)  =  1.000
    C(k, xc)  = -0.997
    C(a, k)   = -0.996
    C(y0, k)  =  0.996
    C(y0, xc) = -0.987
    C(a, y0)  = -0.985


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 36
    # data points      = 14
    # variables        = 4
    chi-square         = 1.9823e-04
    reduced chi-square = 1.9823e-05
    Akaike info crit   = -148.312240
    Bayesian info crit = -145.756011
[[Variables]]
    a:   0.14138171 +/- 0.00308698 (2.18%) (init = 0.13)
    y0:  0.36150581 +/- 0.00202181 (0.56%) (init = 0.36)
    k:   0.47245737 +/- 0.04046693 (8.57%) (init = 0.4)
    xc:  59.6371296 +/- 0.19962892 (0.33%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.689
    C(a, k)   = -0.441
    C(y0, xc) =  0.379
    C(y0, k)  =  0.289


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
key =  "GdmCl_6M"

fit, params, params_error = ana.lm_fit(data_all[key].absorb_df.loc[:,:67], guess=[0.13, 0.36, 0.4])
melt_T[key] = params['xc']
error_T[key] = params_error
residual[key] = [1 - (np.sum(fit["residual"]**2)/np.sum((fit[260] - fit[260].mean(axis=0))**2))]
plot.mult_func([260, "fit"], [fit], error={"fit": fit["error"]}, swap=True, 
x_label="Temperature [°C]", y_label="Absorbance [%]", label=["data", "sigmoid fit", 'melting temperature'], marker=['o', ''], linestyle=[" ", "-"],
subtitle=["Hyperchromatic shift"], title=key, vertical_line=[params['xc']])

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 62
    # data points      = 11
    # variables        = 4
    chi-square         = 1.4727e-04
    reduced chi-square = 2.1038e-05
    Akaike info crit   = -115.432890
    Bayesian info crit = -113.841309
[[Variables]]
    a:   0.17876816 +/- 0.00313028 (1.75%) (init = 0.13)
    y0:  0.32179511 +/- 0.00230700 (0.72%) (init = 0.36)
    k:   0.94413078 +/- 0.11034248 (11.69%) (init = 0.4)
    xc:  52.3436456 +/- 0.12753258 (0.24%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.750
    C(y0, xc) =  0.367
    C(y0, k)  =  0.348
    C(a, k)   = -0.336
    C(k, xc)  =  0.308
    C(a, xc)  = -0.120


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
x = [0., 0.5, 2., 4., 6.]
y = list(melt_T.values())[0:4]
y.append(None)
print(y)
y_error = list(error_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T
error= {"Gdm2SO4":y_error}

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]
y2_error = [error_T["Control_0M"], error_T["GdmCl_0.5M"], error_T["GdmCl_2M"], error_T["GdmCl_4M"], error_T["GdmCl_6M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T
error["GdmCl"] = y2_error 

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration [M]", y_label="Temperature [°C]", error=error_T)

[64.07343657097545, 65.4725786302147, 65.14363697636958, 66.08313970889418, None]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# TD Calculations


<div class="alert alert-block alert-info">
<b>Info:</b> Steps after Mergny, J.-L., & Lacroix, L. (2003). Analysis of Thermal Melting Curves. In OLIGONUCLEOTIDES (Vol. 13). www.liebertpub.com
</div>

**Hypochromatic Shift at 260nm  wit Low and High Plateau Fit**

In [8]:
raw = data_all["Control_0M"].absorb_df
low_fit, params_lowfit = ana.lm_fit(raw[[20,30,40]], f_type='linear', guess=[0.0001, 0.35])
high_fit, params_highfit = ana.lm_fit(raw[[70, 80,90]], f_type='linear', guess=[0.0001, 0.48])
sig_fit, params_sigfit = ana.lm_fit(raw, f_type='s1', guess=[0.2, 0.35, 0.5, 63])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 3.7600e-07
    reduced chi-square = 3.7600e-07
    Akaike info crit   = -43.6768616
    Bayesian info crit = -45.4796371
[[Variables]]
    m:   9.8400e-05 +/- 4.3359e-05 (44.06%) (init = 0.0001)
    y0:  0.35534533 +/- 0.00134808 (0.38%) (init = 0.35)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.965
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.3374e-05
    reduced chi-square = 1.3374e-05
    Akaike info crit   = -32.9623650
    Bayesian info crit = -34.7651404
[[Variables]]
    m:   7.5960e-04 +/- 2.5860e-04 (34.04%) (init = 0.0001)
    y0:  0.42851300 +/- 0.02079508 (4.85%) (init = 0.48)
[[Correlations]] (unreported correlations are <

**Calculating Fraction Folded with High Plateu adjusted manually**

In [9]:
data_treated = pd.DataFrame(raw.loc[260])
data_treated["low_fit"] = ana.linear(np.array(raw.columns), *params_lowfit.values())
data_treated["high_fit"] = ana.linear(np.array(raw.columns), 0.0004495999948082116, 0.45451300041966917)
data_treated["intermediate"] = 0.5*(data_treated["low_fit"] + data_treated["high_fit"])
data_treated["fit"] = sig_fit["fit"]
data_treated["baseline_fit"] = (data_treated["high_fit"] - data_treated[260])/(data_treated["high_fit"] - data_treated["low_fit"])
data_treated["normed_data"] = 1- ana.normalize(raw).loc[260].to_numpy()
data_treated["log_fit"] = 1 - ana.normalize(data_treated)["fit"]
plot.mult_func([260, "low_fit", "high_fit", "intermediate", "fit"], [data_treated.T], subtitle=[""], title="Hypochromatic shift with Plaetau Fits", y_label="Absorbance at 260nm")
plot.mult_func(["baseline_fit", "normed_data", "log_fit"], [data_treated.T], subtitle=[""],  title="Fraction folded calculated by Plateu fits and Normalization", y_label="Fraction folded")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**calulate K and ln K for different methods**

In [10]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
base_method = pd.DataFrame(data_treated["baseline_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
base_method['theta'] = base_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
base_method["K"] = (base_method["theta"])/(c_total*((1 - base_method["theta"])**2))
base_method['ln K'] = np.log(base_method['K'])

# select values in confidence values 0.1 < theta < 0.9
base_method  = base_method.loc[60:70]
# change celsius to kelvin
base_method = base_method.set_index(1/(base_method.index + 273.15))

# make linear fit and plot
enthalpy_base, params_base = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_base["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_base.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
normed_method = pd.DataFrame(data_treated["normed_data"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
normed_method['theta'] = normed_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
normed_method["K"] = (normed_method["theta"])/(c_total*((1 - normed_method["theta"])**2))
normed_method['ln K'] = np.log(normed_method['K'])

# select values in confidence values 0.1 < theta < 0.9
normed_method  = normed_method.loc[60:70]
# change celsius to kelvin
normed_method = normed_method.set_index(1/(normed_method.index + 273.15))

# make linear fit and plot
enthalpy_normed, params_norm = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_normed["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_normed.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
fit_method = pd.DataFrame(data_treated["log_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
fit_method['theta'] = fit_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
fit_method["K"] = (fit_method["theta"])/(c_total*((1 - fit_method["theta"])**2))
fit_method['ln K'] = np.log(fit_method['K'])

# select values in confidence values 0.1 < theta < 0.9
fit_method  = fit_method.loc[60:70]
# change celsius to kelvin
fit_method = fit_method.set_index(1/(fit_method.index + 273.15))

# make linear fit and plot
enthalpy_fit, params_fit = ana.lm_fit(fit_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_fit["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_fit.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5
    # variables        = 2
    chi-square         = 1.62690318
    reduced chi-square = 0.54230106
    Akaike info crit   = -1.61379796
    Bayesian info crit = -2.39492213
[[Variables]]
    m:   143015.744 +/- 10592.4844 (7.41%) (init = 80000)
    y0: -405.548597 +/- 31.3671034 (7.73%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Calculate Entropy and Enthalpy**

In [13]:
enthalpy = pd.DataFrame(["H [kJ/mol]", "H [kcal/mol]", "S [kJ/mol*K]", "S [kcal/mol*K]"], columns=["units"])
enthalpy = enthalpy.set_index("units")

# for base method
m, y0 = params_base.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["base"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for normed method
m, y0 = params_norm.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["norm"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for fit method
m, y0 = params_fit.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["fit"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

print(enthalpy)

                      base        norm          fit
units                                              
H [kJ/mol]     -686.554867 -686.554867 -1189.104403
H [kcal/mol]   -164.090527 -164.090527  -284.202730
S [kJ/mol*K]     -1.887004   -1.887004    -3.371934
S [kcal/mol*K]   -0.451005   -0.451005    -0.805911


## Derivative


In [14]:
deriv = ana.derivative(data_all["Control_0M"].absorb_df)
data = data_all["Control_0M"].absorb_df

In [15]:
plot.mult_func([260], [data], [deriv], title= "Hypochromatic shift", subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Parafak Ethanol

## *setting up the Notebook*

In [9]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib

from scipy import integrate
import lmfit
import os

In [10]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [11]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [12]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [13]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Ethanol"
datalist = os.listdir(path)
print(datalist)

['Control_0%', 'Ethanol_50%', 'Ethanol_50%_nachTest', 'Ethanol_50%_real', 'Ethanol_50%_vorTest']


In [14]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [15]:
df =  data_all["Control_0%"].cd_df
df = df.rename(columns={-1:"0%"})
df["50%"] =  (data_all["Ethanol_50%"].cd_df.loc[:,-1])
df["nach"] =  (data_all["Ethanol_50%_nachTest"].cd_df.loc[:,-1])
df["vor"] =  (data_all["Ethanol_50%_vorTest"].cd_df.loc[:,-1])
df["50% real"] =  (data_all["Ethanol_50%_real"].cd_df.loc[:,-1])
print(df)

                  0%       50%      nach       vor  50% real
wavelength                                                  
220         2.018260  2.274510  3.126240  2.307520  2.848860
221         2.080980  2.351290  3.314980  2.367540  2.935160
222         2.050420  2.385820  3.392230  2.399380  2.992490
223         1.912300  2.286210  3.399130  2.399530  2.952990
224         1.639410  2.188990  3.234820  2.326470  2.905010
225         1.334610  2.006050  2.974400  2.194240  2.709800
226         0.987198  1.847160  2.640560  1.936590  2.520000
227         0.635405  1.639230  2.248080  1.665910  2.251200
228         0.240217  1.421670  1.923790  1.448930  1.967390
229        -0.119267  1.181210  1.646120  1.259320  1.597030
230        -0.512917  0.921575  1.261840  1.015550  1.176200
231        -0.886912  0.619775  0.926121  0.713306  0.785832
232        -1.336630  0.354009  0.370603  0.331813  0.390593
233        -1.776360  0.074562  0.018606  0.083282  0.037718
234        -2.209810 -0.

In [16]:
plot.mult_func(['0%', 'vor', 'nach', '50%', "50% real" ], [df], swap=True, marker=["", "", "", "", ""], x_label="Wavelengt [nm]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …